# FSL Project Part 1
Feature Extraction, Density Estimation and Bayesian Classification

In [1]:
''' Imports '''
import scipy.io
from scipy.spatial import distance
import numpy as np
from numpy.linalg import inv, det
import math

In [2]:
def getNormalizedFeatures(data_arr):
    ''' Calculate mean and std dev of dataset '''
    n_imgs = data_arr.shape[0]
    # print(n_imgs)
    m_arr = np.zeros(n_imgs)
    s_arr = np.zeros(n_imgs)

    for i in range(n_imgs):
        m_arr[i] = np.mean(data_arr[i])
        s_arr[i] = np.std(data_arr[i])
        
    ''' Calculate mean and std dev of the means and std devs respectively '''
    M1 = np.mean(m_arr)
    M2 = np.mean(s_arr)
    S1 = np.std(m_arr)
    S2 = np.std(s_arr)
    ''' Create array of normalized vectors '''
    y = np.zeros((n_imgs, 2))

    for i in range(n_imgs):
        y[i] = [(m_arr[i]-M1)/S1, (s_arr[i]-M2)/S2]
    
    return np.transpose(y)

In [3]:
def discrim(x, mean, cov, prior):
    maha = distance.mahalanobis(x, mean, inv(cov))
    expFac = math.exp(-0.5 * (maha**2))
    const = 2 * math.pi * math.sqrt(det(cov))
    like = expFac * prior / const
    return like

In [4]:
''' Import training dataset '''
train_data = scipy.io.loadmat('train_data.mat')
# print(data)

train_data_arr = np.array(train_data['data'])
label_arr = np.array(train_data['label'][0])
# print(dataset.shape)
# print(dataset[0][10])
# print(label_arr)

n_train = train_data_arr.shape[0]

In [5]:
''' Import test dataset '''
test_data = scipy.io.loadmat('test_data.mat')
# print(test_data)
test_data_arr = np.array(test_data['data'])
test_label_arr = np.array(test_data['label'])[0]
# print(test_data_arr.shape)
# print(test_label_arr.shape)

n_test = test_data_arr.shape[0]
# print(n_test)

In [6]:
train_y = getNormalizedFeatures(train_data_arr)
# print(train_y[0:3])

In [7]:
test_y = getNormalizedFeatures(test_data_arr)
# print(test_y[0:3])

In [8]:
''' Split features into 3 and 7 classes '''
feature_3 = []
feature_7 = []

train_y = np.transpose(train_y)
test_y = np.transpose(test_y)
for i in range(n_train):
    if label_arr[i] == 3:
        feature_3.append(train_y[i])
    else:
        feature_7.append(train_y[i])

feature_3 = np.array(feature_3)
feature_7 = np.array(feature_7)
# print(feature_3[0])
# print(len(feature_7))

In [9]:
mu_3 = np.mean(feature_3, 0)
mu_7 = np.mean(feature_7, 0)
# print(mu_3)
# print(mu_7)

''' Calculate variance '''
len_3 = len(feature_3)
sum_3 = 0
for i in range(len_3):
    mat = (feature_3[i] - mu_3).reshape(2,1)
    trans = mat.reshape(1,2)
    sum_3 += mat @ trans
sigma_3 = (1/len_3)*sum_3

len_7 = len(feature_7)
sum_7 = 0
for i in range(len_7):
    mat = (feature_7[i] - mu_7).reshape(2,1)
    trans = mat.reshape(1,2)
    sum_7 += mat @ trans
sigma_7 = (1/len_7)*sum_7

# print(sigma_3)
# print(sigma_7)

In [11]:
''' Training Case 1 '''
train_case1_output = np.zeros(n_train)
train_error_case1 = np.zeros(n_train)
for i in range(n_train):
    l3 = discrim(train_y[i], mu_3, sigma_3, 0.5)
    l7 = discrim(train_y[i], mu_7, sigma_7, 0.5)
    if l3 > l7:
        train_case1_output[i] = 3
        train_error_case1[i] = l7
    else:
        train_case1_output[i] = 7
        train_error_case1[i] = l3

print("Training Data Case 1: Error probability is {}".format(np.mean(train_error_case1)))

''' Training Case 2 '''
train_case2_output = np.zeros(n_train)
train_error_case2 = np.zeros(n_train)
for i in range(n_train):
    l3 = discrim(train_y[i], mu_3, sigma_3, 0.3)
    l7 = discrim(train_y[i], mu_7, sigma_7, 0.7)
    if l3 > l7:
        train_case2_output[i] = 3
        train_error_case2[i] = l7
    else:
        train_case2_output[i] = 7
        train_error_case2[i] = l3
        
print("Training Data Case 2: Error probability is {}".format(np.mean(train_error_case2)))

Training Data Case 1: Error probability is 0.15844983966530024
Training Data Case 2: Error probability is 0.10538315677294734


In [12]:
''' Test Case 1 '''
test_case1_output = np.zeros(n_test)
test_error_case1 = np.zeros(n_test)
for i in range(n_test):
    l3 = discrim(test_y[i], mu_3, sigma_3, 0.5)
    l7 = discrim(test_y[i], mu_7, sigma_7, 0.5)
    if l3 > l7:
        test_case1_output[i] = 3
        test_error_case1[i] = l7
    else:
        test_case1_output[i] = 7
        test_error_case1[i] = l3
        
print("Test Data Case 1: Error probability is {}".format(np.mean(test_error_case1)))

''' Test Case 2 '''
test_case2_output = np.zeros(n_test)
test_error_case2 = np.zeros(n_test)
for i in range(n_test):
    l3 = discrim(test_y[i], mu_3, sigma_3, 0.3)
    l7 = discrim(test_y[i], mu_7, sigma_7, 0.7)
    if l3 > l7:
        test_case2_output[i] = 3
        test_error_case2[i] = l7
    else:
        test_case2_output[i] = 7
        test_error_case2[i] = l3
        
print("Test Data Case 2: Error probability is {}".format(np.mean(test_error_case2)))

Test Data Case 1: Error probability is 0.15900897734657957
Test Data Case 2: Error probability is 0.10540420592913254
